In [1]:
import os
import sys
import cv2
import random
import itertools
import numpy as np
from collections import Counter
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans
from math import ceil

def get_features(path):
    features = []
    names = []
    skip = 0
    for autor in os.listdir(path):
        segments_path = os.path.join(path, autor)
        segment_number += len(segments_path)
        for segment in os.listdir(segments_path):
            segment_path = os.path.join(segments_path, segment)
            image = cv2.imread(segment_path)
            if np.sum(image) == 0:
                skip += 1
                continue
            features.append(get_features(image))
            names.append(segment_path)
    return features, np.array(names)

def get_clusters(most_100, names, kernel_size = 13):
    start = most_100[0]
    clusters = []
    clusters_images = []
    for i in most_100:
        segments_name = np.ravel(names[np.argwhere(clust == i)])
        line = cv2.imread(segments_name[0])
        images = [line]
        class_segments = [get_features(line)]
        for seg_path in segments_name[1:]:
            img = cv2.imread(seg_path)
            class_segments.append(get_features(img))
            line = np.hstack((line, img))
            images.append(img)
        clusters.append(class_segments)
        clusters_images.append(images)
    return clusters, clusters_images
    
def find_closest_feature_vector(features, images):
    dists = np.zeros((len(features), len(features)))
    for (i, j) in itertools.product(range(len(features)), range(len(features))):
            dists[i, j] = euclidean(features[i], features[j])
    min_ind = np.argmin(np.min(dists, axis=1))
    return features[min_ind], images[min_ind]
 
if __name__ == "__main__": 
    path ='C:/Users/Anastasia/Pictures/autors_segments'
    features, names = get_features(path)
    kmeans = KMeans(n_clusters=150, random_state=0).fit(features)
    clust = kmeans.labels_
    max_cluster_size = Counter(clust).most_common(1)[0][1]
    most_100 = [cluster for (cluster, count) in Counter(clust).most_common(200)[10:110]]

    clusters, clusters_images = get_clusters(most_100, names)

    cluster_centres = []
    for clust, images in zip(clusters, clusters_images):
        feature, image = find_closest_feature_vector(clust, images)
        cluster_centres.append(feature)
         
    cluster_centres = np.array(cluster_centres)
    np.save('cluster_centres_kernel_13', cluster_centres)

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'C:/Users/Anastasia/Pictures/autors_segments'

In [2]:
import os
import cv2
import numpy as np

import SegmentWords
import SegmentFeatures
from scipy.spatial.distance import euclidean

def dist(vector, features):
    distances = []
    for feature in features:
        distances.append(euclidean(vector, feature))
    distances = np.array(distances)
    return np.argmin(distances)
    
    
def transform(str_numb):
    l, r = str_numb.split("_")
    return float(l + "." + r)
    
def get_str2numb_numb2dict(vect):
    str_to_ind_dict = {}
    count = 0
    for v in vect:
        if v not in str_to_ind_dict.keys():
            str_to_ind_dict[v] = count
            count += 1
    reverse_dict = {v:k for k, v in str_to_ind_dict.items()}
    return str_to_ind_dict, reverse_dict

def apply_dict(dict_keys, X):
    res = []
    for x in X:
        res.append(dict_keys[x])
    return res
    
def compute_features(train_path, save_name_x, save_name_y):
    train_features = []
    train_y = []
    for variant in os.listdir(train_path):
        v = variant[-1]
        autors_path = os.path.join(train_path, variant)
        autors = os.listdir(autors_path)
        for autor in autors:
            train_y.append(str(v) + "_" + str(autor))
            words = os.listdir(os.path.join(autors_path, autor))
            path = os.path.join(autors_path, autor)
            autor_feature =  np.zeros((len(cluster_centres)))
            for word in words:
                word_path = os.path.join(path, word)
                image = cv2.imread(word_path)
                segments = SegmentWords.get_segments(image, kernel_size = 11)
                for segment in segments:
                    if np.sum(segment == 0) > 20  and (255 in segment):
                        feature = SegmentFeatures.get_features(np.stack((segment,)*3, axis=-1))
                        autor_feature[dist(feature, cluster_centres)] += 1
            train_features.append(autor_feature / max(autor_feature))
            
    np.save(save_name_x, train_features)
    np.save(save_name_y, train_y)
    return train_features, train_y
    
    

ModuleNotFoundError: No module named 'SegmentWords'

In [3]:

from scipy.spatial.distance import euclidean
import os
import sys
import cv2
import random
import itertools
import numpy as np
from collections import Counter
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans
from math import ceil

In [4]:
def get_contour(image):
    img = image[:,:,0]
    ret, thresh = cv2.threshold(img,127,255,0)
    _, contours, hierarchy = cv2.findContours(thresh, 1, 2)
    return contours[0]

def horizontal_hist(image):
    img = image[:,:,0]
    return list(np.sum(img / 255, axis = 0))

def vertical_hist(image):
    img = image[:,:,0]
    return list(np.sum(img / 255, axis = 1))

def upper_profile(image):
    img = image[:,:,0]
    black = np.argwhere(img == 0)
    ind = min(black[:,0])
    return list(img[ind, :] / 255)
    
def lower_profile(image):
    img = image[:,:,0]
    black = np.argwhere(img == 0)
    ind = max(black[:,0])
    return list(img[ind, :] / 255)

def orientation(image):
    cnt = get_contour(image)
    if len(cnt) < 5:
        return 0
    (x,y), (MA,ma), angle = cv2.fitEllipse(cnt)
    return angle
       
def rectangularity(image):
    cnt = get_contour(image)
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w * h
    return float(area) / rect_area
    
def solidity(image):
    cnt = get_contour(image)
    area = cv2.contourArea(cnt)
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    if hull_area == 0:
        hull_area = 1
    return float(area) / hull_area
    
def eccentricity(image):
    img = image[:,:,0]
    ret, thresh = cv2.threshold(img, 127, 255, 0)
    cnt = get_contour(image)
    area = cv2.contourArea(cnt)
    moments = cv2.moments(thresh, 1)
    if area == 0:
        area = 1
    return (((moments['m02'] - moments['m02']) ** 2) + 4 * moments['m11']) / area

def elongation(image):
    img = image[:,:,0]
    cnt = get_contour(image)
    area = cv2.contourArea(cnt)
    kernel = np.ones((3,3), np.uint8)
    iteration_count = 1
    erosion = cv2.erode(img, kernel, iterations = 1)
    while 255 in erosion:
        erosion = cv2.erode(erosion, kernel, iterations = 1)
        iteration_count += 1
    return area / (2 * iteration_count * iteration_count)
    
def perimeter(image):
    cnt = get_contour(image)
    perimeter = cv2.arcLength(cnt, True)
    return perimeter

def get_features(image):
    feature = horizontal_hist(image)
    feature += vertical_hist(image)
    feature += upper_profile(image)
    feature += lower_profile(image)
    feature.append(orientation(image))
    feature.append(rectangularity(image))
    feature.append(solidity(image))
    feature.append(eccentricity(image))
    feature.append(elongation(image))
    feature.append(perimeter(image))
    feature = np.array(feature)
    return feature / np.max(feature)

In [5]:
import itertools
import numpy as np

def get_start_(image):
    black_pixels = np.argwhere(image == 0)
    max_x = max(black_pixels[:,0])
    pixels_with_max_x = black_pixels[np.where(black_pixels[:,0] == max_x), :][0]
    min_y = min(pixels_with_max_x[:,1])
    return np.ravel(pixels_with_max_x[np.where(pixels_with_max_x[:,1] == min_y), :][0])

def get_start(image):
    black_pixels = np.argwhere(image == 0)
    min_y = min(black_pixels[:,1])
    pixels_with_min_y = black_pixels[np.where(black_pixels[:,1] == min_y), :][0]
    max_x = max(pixels_with_min_y[:,0])
    return np.ravel(pixels_with_min_y[np.where(pixels_with_min_y[:,0] == max_x), :][0])


def get_segments(image, kernel_size = 15):
    img = image.copy()[:,:,0]
    n, m = img.shape
    ret, thresh = cv2.threshold(img, 127, 255, 0)
    img = cv2.bitwise_not(thresh)     
    _, markers = cv2.connectedComponents(img)
    component_count = np.amax(markers)
    segments = []    
    for comp in range(1, component_count + 1):
        mask = markers == comp
        component = img * mask
        if np.sum(component) / 255 < 100:
            continue
        component = cv2.bitwise_not(component)
        start = get_start(component)     
        start_points = [start]
        used_cells = []
        segments_points = []
        component_3 = np.stack((component,)*3, axis=-1)
        while len(start_points) > 0:
            start = start_points.pop(0)
            if (0 < start[0] - kernel_size < n):
                top_left = (start[1], start[0] - kernel_size)
            elif start[0] - kernel_size >= n:
                top_left = (start[1], n)
            elif start[0] - kernel_size <= 0:
                top_left = (start[1], 0)
                
            if (0 < start[1] + kernel_size < m):
                bottom_right = (start[1] + kernel_size, start[0])
            elif start[1] + kernel_size >= m:
                bottom_right = (m, start[0])
            elif start[1] + kernel_size <= 0:
                bottom_right = (0, start[0])

                
            if top_left is None or bottom_right is None:
                continue

            segments_points.append([top_left, bottom_right])
            top = max(0, start[0] - kernel_size)
            bottom = min(n, start[0])
            left = max(0, start[1])
            right = min(m, start[1] + kernel_size)

            segment = component[top:bottom, left:right]
            s_n, s_m = segment.shape
            if s_n < kernel_size:
                segment = cv2.copyMakeBorder(segment,kernel_size - s_n,0,0,0,cv2.BORDER_CONSTANT,value=255)
            if s_m < kernel_size:
                segment = cv2.copyMakeBorder(segment,0,0,0,kernel_size - s_m,cv2.BORDER_CONSTANT,value=255)

            segments_cells = list(itertools.product(range(top, bottom + 1), range(left, right + 1)))
            if not set(segments_cells).issubset(set(used_cells)): 
                used_cells += segments_cells
                segments.append(segment)
            else: 
                continue

            offset = 0
            if (0 <= start[0] - kernel_size - 1 <= n) and (0 <= start[1] + kernel_size <= m):
                top_line = component[top - 1,  left:right]   
                if 0 in top_line:
                    new_start = (start[0] - kernel_size - 1, start[1] + offset)
                    if new_start not in used_cells and new_start not in start_points:
                        start_points.append(new_start)

            if (0 <= start[0] +  kernel_size + 1 <= n) and (0 <= start[1] + kernel_size <= m):
                if bottom + 1 != n:
                    bot_line = component[bottom + 1,  left:right]
                    if 0 in bot_line:
                        new_start = (start[0] + kernel_size + 1, start[1] + offset)
                        if new_start not in used_cells and new_start not in start_points:
                            start_points.append(new_start)

            if (0 <= start[0] - kernel_size <= n) and (0 <= start[1] - kernel_size - 1 <= m):
                left_line = component[top: bottom,  left - 1] 
                if 0 in left_line:
                    new_start = (start[0] - offset, start[1] - kernel_size - 1)
                    if new_start not in used_cells and new_start not in start_points:
                        start_points.append(new_start)

            if (0 <= start[0] - kernel_size <= n) and (0 <= start[1] + kernel_size + 1 <= m):
                if right + 1 != m:   
                    right_line = component[top: bottom,  right + 1]
                    if 0 in right_line:
                        new_start = (start[0] - offset, start[1] + kernel_size + 1)
                        if new_start not in used_cells and new_start not in start_points:
                            start_points.append(new_start)

    return segments
    
    
def save_segments(path, segments, count = 0):
    for segment in segments:
        if np.sum(segment == 0) > 10:
            cv2.imwrite(os.path.join(path , 'segment_' + str(count) + '.png'), segment)
            count += 1
           

def get_starts(vect):
    inds = []
    for i, val in enumerate(vect):
        if val == 0 and i == 0:
            inds.append(i)
        if i > 0 and vect[i - 1] == 255 and val == 0:
            inds.append(i)
    return inds


In [6]:
import numpy as np
from sklearn import svm
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier

In [7]:
def get_str2numb_numb2dict(vect):
    str_to_ind_dict = {}
    count = 0
    for v in vect:
        if v not in str_to_ind_dict.keys():
            str_to_ind_dict[v] = count
            count += 1
    reverse_dict = {v:k for k, v in str_to_ind_dict.items()}
    return str_to_ind_dict, reverse_dict

def apply_dict(dict_keys, X):
    res = []
    for x in X:
        res.append(dict_keys[x])
    return res

def arithmetic_round(x):
    a = int(x)
    b = x - a
    if (b < 0.5): 
        return round(x)
    else:
        return round(x + 0.01)

In [8]:
def identify_knn(X, y, test_features, test_y, ind_to_str_dict, metric='chebyshev'):
    neigh = KNeighborsClassifier(n_neighbors=2, metric=metric)
    neigh.fit(X, y) 
    res = neigh.predict(test_features) 
    predict = apply_dict(ind_to_str_dict, res)
    count = 0
    for i,j in zip(predict, test_y):
        if i == j:
            count += 1

    print("top-1 autor accuracy: {}%".format(arithmetic_round(100 * count / len(test_y))))

    predict = neigh.predict_proba(test_features)
    count = 0
    for i, res in enumerate(predict):
        ind = np.argpartition(res, -4)[-4:]
        ind = apply_dict(ind_to_str_dict , ind)
        if test_y[i] in ind:
            count += 1

    print("top-5 autor accuracy: {}%".format(arithmetic_round(100 * count / len(test_y))))